### 1. rpyc for MPC

In [3]:
import rpyc
import labbench as lb
from labbench import paramattr as attr
from typing import Type
import multiprocessing
import numpy as np
import ctypes
from multiprocessing import RawArray, shared_memory

N = int(25e6*0.2)


class MyDevice(lb.Device):
    size: int = attr.value.int(default=500_000_000, min=1)
    backend = None

    def open(self):
        print('open!')
        d_size = np.dtype(np.float32).itemsize * N
        name = f'device-{hex(id(self))}'
        try:
            self._shm = shared_memory.SharedMemory(create=True, size=d_size, name=name)
        except FileExistsError:
            self._shm = shared_memory.SharedMemory(create=False, size=d_size, name=name)

            if self._shm.size < d_size:
                self._shm.close()
                self._shm = shared_memory.SharedMemory(create=True, size=d_size, name=name)
        

        # self.shared_array = RawArray(ctypes.c_double, N)
        self.x = np.ndarray(N, dtype=np.float32, buffer=self._shm.buf)

    def close(self):
        print('close!')
        try:
            self._shm.close()
        except BufferError:
            pass

    def big_output(self):
        self.x[:10] = 5
        return self.x

    def donothing(self):
        pass


class SubprocessDeviceAdapter:
    conn: rpyc.core.protocol.Connection
    device: lb.Device

    def __init__(self, conn, device):
        self.conn = conn
        self.device = device

    def open(self):
        # conn is already open
        self.device.open()
        return self.device

    def close(self):
        try:
            if self.device is not None:
                self.device.close()
        finally:
            self.conn.close()

    def __enter__(self, *args, **kws):
        print('adapter enter')
        self.open()
        return self.device

    def __exit__(self, *args, **kws):
        print('adapter exit')
        self.close()


class DeviceService(rpyc.Service):
    def __init__(self, radio_type: Type[MyDevice], *args, **kws):
        self.device = radio_type(*args, **kws)


@lb.retry(ConnectionRefusedError, tries=5)
def connect(*args):
    return rpyc.connect(*args)


def spawn_device(cls: lb.Device, *args, **kws):
    """return a context manager that opens `cls` instantiated with the given arguments in another process """

    ctx = multiprocessing.get_context('fork')
    service = rpyc.utils.helpers.classpartial(DeviceService, cls, *args, **kws)
    conf = {'allow_all_attrs': True, 'logger': lb.logger}
    svc = rpyc.OneShotServer(service=service, protocol_config=conf)
    ctx.Process(target=svc.start).start()

    lb.sleep(0.25)
    conn = connect('localhost', svc.port)
    return SubprocessDeviceAdapter(conn, conn.root.device)

def array_from_server(shm_netref, x: np.ndarray):
    shm = shared_memory.SharedMemory(create=False, size=device._shm.size, name=device._shm.name)
    return np.frombuffer(shm.buf, dtype=str(x.dtype)).reshape(x.shape)

with spawn_device(MyDevice, size=4) as device, lb._host.Host() as host:
    import cupy
    device._logger.info('message!')
    x = device.big_output()
    # x = np.ndarray(device.x.shape, dtype=np.float32, buffer=shared)
    %timeit -n1 -r1 cupy.array(array_from_server(device._shm, x)).get()
    %timeit -n1 -r1 cupy.array(array_from_server(device._shm, x)).get()
    y = array_from_server(device._shm, x)
    del x,y


open!


 INFO   2024-07-31 09:38:10,732.732 • MyDevice(): message!
Exception ignored in: <function SharedMemory.__del__ at 0x7f67dacdc0>
Traceback (most recent call last):
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 184, in __del__
    self.close()
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 227, in close
    self._mmap.close()
BufferError: cannot close exported pointers exist


adapter enter


Exception ignored in: <function SharedMemory.__del__ at 0x7f67dacdc0>
Traceback (most recent call last):
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 184, in __del__
    self.close()
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 227, in close
    self._mmap.close()
BufferError: cannot close exported pointers exist
Exception ignored in: <function SharedMemory.__del__ at 0x7f67dacdc0>
Traceback (most recent call last):
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 184, in __del__
    self.close()
  File "/home/dkuester/micromamba/envs/flex-spectrum-sensor/lib/python3.9/multiprocessing/shared_memory.py", line 227, in close
    self._mmap.close()
BufferError: cannot close exported pointers exist


close!
118 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
47.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
adapter exit


In [4]:
0.2+.24+.05

0.49

In [38]:
d_size = np.dtype(np.float32).itemsize * N
_shm = shared_memory.SharedMemory(create=False, size=d_size, name=f'device-')

# self.shared_array = RawArray(ctypes.c_double, N)
x = np.ndarray(N, dtype=np.float32, buffer=_shm.buf)
y = np.ndarray(N, dtype=np.float32, buffer=x.base)

In [35]:
dir(x.base)

<mmap.mmap closed=False, access=ACCESS_DEFAULT, length=200000000, pos=0, offset=0>

In [33]:
import mmap
mmap.mmap(fileno, length, tagname=None, access=ACCESS_DEFAULT[, offset])
dir(x.base)

['__class__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'closed',
 'find',
 'flush',
 'madvise',
 'move',
 'read',
 'read_byte',
 'readline',
 'resize',
 'rfind',
 'seek',
 'size',
 'tell',
 'write',
 'write_byte']

In [36]:
x.base.fileno

AttributeError: 'mmap.mmap' object has no attribute 'fileno'

In [17]:
[n for n in dir(y) if 'mem' in n]

['__cuda_memory__']

In [20]:
y.get_ipc_handle()

CudaAPIError: [801] Call to cuIpcGetMemHandle results in CUDA_ERROR_NOT_SUPPORTED

### 2. RPYC for remote control

In [ ]:
from rpyc.utils.zerodeploy import DeployedServer
from plumbum import SshMachine

# create the deployment
mach = SshMachine("somehost", user="someuser", keyfile="/path/to/keyfile")
server = DeployedServer(mach)

# and now you can connect to it the usual way
conn1 = server.classic_connect()
print(conn1.modules.sys.platform)

# you're not limited to a single connection, of course
conn2 = server.classic_connect()
print(conn2.modules.os.getpid())

# when you're done - close the server and everything will disappear
server.close()
